## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import numpy as np

In [2]:
#  Import and read the charity_data.csv.
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
column_reduce_df = application_df.drop(columns=["EIN", "NAME"])
column_reduce_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Determine the number of unique values in each column.
print(column_reduce_df.nunique())

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [5]:
# Look at APPLICATION_TYPE value counts for binning
column_reduce_df["APPLICATION_TYPE"].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = column_reduce_df['APPLICATION_TYPE'].value_counts().loc[lambda x : x<200].index
application_types_to_replace

Index(['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17'], dtype='object')

In [7]:
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts for binning
column_reduce_df["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [9]:
# You may find it helpful to look at CLASSIFICATION value counts >1
column_reduce_df['CLASSIFICATION'].value_counts().loc[lambda x : x>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [10]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = column_reduce_df['CLASSIFICATION'].value_counts().loc[lambda x : x<1000].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [11]:
# Convert categorical data to numeric with `pd.get_dummies`
data_df = pd.get_dummies(data=column_reduce_df, columns=["APPLICATION_TYPE", "AFFILIATION", "CLASSIFICATION", "USE_CASE", "ORGANIZATION", "INCOME_AMT", "SPECIAL_CONSIDERATIONS"])
data_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [12]:
# Split our preprocessed data into our features and target arrays
X = data_df.drop(columns="IS_SUCCESSFUL")
y = data_df["IS_SUCCESSFUL"]
X.head()

,STATUS,ASK_AMT,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [13]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [14]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [15]:
type(y_train)

pandas.core.series.Series

In [16]:
# Convert y_train and y_test to numpy arrays
y_train_array = np.asarray(y_train)
y_test_array = np.asarray(y_test)

## Model optimization attempt #1:
#### Add a third hidden layer

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn1 = tf.keras.models.Sequential()

nn1.add(tf.keras.layers.Dense(units=116, activation="relu", input_dim=116))

# First hidden layer
nn1.add(tf.keras.layers.Dense(units=58, activation="relu"))

# Second hidden layer
nn1.add(tf.keras.layers.Dense(units=29, activation="relu"))

# Third hidden layer
nn1.add(tf.keras.layers.Dense(units=14, activation="relu"))

# Output layer
nn1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 116)               13572     
_________________________________________________________________
dense_1 (Dense)              (None, 58)                6786      
_________________________________________________________________
dense_2 (Dense)              (None, 29)                1711      
_________________________________________________________________
dense_3 (Dense)              (None, 14)                420       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 15        
Total params: 22,504
Trainable params: 22,504
Non-trainable params: 0
_________________________________________________________________


2021-12-24 10:40:53.674229: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-24 10:40:53.675785: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.


In [18]:
# Compile the model
nn1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
# Train the model
fit_model1 = nn1.fit(X_train_scaled, y_train_array, epochs=150)

Train on 25724 samples
Epoch 1/150
25724/25724 [==============================] - 6s 231us/sample - loss: 0.5692 - accuracy: 0.7219
Epoch 2/150
25724/25724 [==============================] - 4s 155us/sample - loss: 0.5510 - accuracy: 0.7303
Epoch 3/150
25724/25724 [==============================] - 4s 174us/sample - loss: 0.5477 - accuracy: 0.7326
Epoch 4/150
25724/25724 [==============================] - 4s 159us/sample - loss: 0.5469 - accuracy: 0.7327
Epoch 5/150
25724/25724 [==============================] - 5s 186us/sample - loss: 0.5451 - accuracy: 0.7352
Epoch 6/150
25724/25724 [==============================] - 4s 142us/sample - loss: 0.5439 - accuracy: 0.7352
Epoch 7/150
25724/25724 [==============================] - 3s 133us/sample - loss: 0.5429 - accuracy: 0.7353
Epoch 8/150
25724/25724 [==============================] - 4s 143us/sample - loss: 0.5421 - accuracy: 0.7353
Epoch 9/150
25724/25724 [==============================] - 3s 132us/sample - loss: 0.5425 - accuracy: 0.7

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn1.evaluate(X_test_scaled,y_test_array,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/1 - 1s - loss: 0.5498 - accuracy: 0.7297
Loss: 0.5968263925338279, Accuracy: 0.72967928647995


### Observation on optimization trial #1
Adding an additional hidden layer lead to worse performance (lower accuracy, higher loss) than the original model.

- - -

## Model optimization attempt #2:
#### Return to only two hidden layers; increase number of nodes in second hidden layer

In [21]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn2 = tf.keras.models.Sequential()

nn2.add(tf.keras.layers.Dense(units=116, activation="relu", input_dim=116))

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=58, activation="relu"))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=58, activation="relu"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 116)               13572     
_________________________________________________________________
dense_6 (Dense)              (None, 58)                6786      
_________________________________________________________________
dense_7 (Dense)              (None, 58)                3422      
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 59        
Total params: 23,839
Trainable params: 23,839
Non-trainable params: 0
_________________________________________________________________


In [22]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [23]:
# Train the model
fit_model1 = nn2.fit(X_train_scaled, y_train_array, epochs=150)

Train on 25724 samples
Epoch 1/150
25724/25724 [==============================] - 6s 214us/sample - loss: 0.5700 - accuracy: 0.7211
Epoch 2/150
25724/25724 [==============================] - 5s 195us/sample - loss: 0.5518 - accuracy: 0.7306
Epoch 3/150
25724/25724 [==============================] - 7s 261us/sample - loss: 0.5481 - accuracy: 0.7324
Epoch 4/150
25724/25724 [==============================] - 6s 239us/sample - loss: 0.5465 - accuracy: 0.7327
Epoch 5/150
25724/25724 [==============================] - 5s 210us/sample - loss: 0.5456 - accuracy: 0.7337
Epoch 6/150
25724/25724 [==============================] - 4s 165us/sample - loss: 0.5443 - accuracy: 0.7344
Epoch 7/150
25724/25724 [==============================] - 4s 166us/sample - loss: 0.5435 - accuracy: 0.7346
Epoch 8/150
25724/25724 [==============================] - 4s 172us/sample - loss: 0.5425 - accuracy: 0.7357
Epoch 9/150
25724/25724 [==============================] - 5s 180us/sample - loss: 0.5415 - accuracy: 0.7

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test_array,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/1 - 1s - loss: 0.5373 - accuracy: 0.7299
Loss: 0.576450124537632, Accuracy: 0.729912519454956


### Observation on optimization trial #2
Increasing the number of nodes on in the second hidden layer from 29 to 58 lead to better performance (slightly higher accuracy, lower loss) than the original model.

- - -

## Model optimization attempt #3:
#### Retain changes made in trial #2; change activation fuction of second hidden layer from relu to tanh

In [25]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn3 = tf.keras.models.Sequential()

nn3.add(tf.keras.layers.Dense(units=116, activation="relu", input_dim=116))

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=58, activation="relu"))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=58, activation="tanh"))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 116)               13572     
_________________________________________________________________
dense_10 (Dense)             (None, 58)                6786      
_________________________________________________________________
dense_11 (Dense)             (None, 58)                3422      
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 59        
Total params: 23,839
Trainable params: 23,839
Non-trainable params: 0
_________________________________________________________________


In [26]:
# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [27]:
# Train the model
fit_model1 = nn3.fit(X_train_scaled, y_train_array, epochs=150)

Train on 25724 samples
Epoch 1/150
25724/25724 [==============================] - 7s 260us/sample - loss: 0.5663 - accuracy: 0.7184
Epoch 2/150
25724/25724 [==============================] - 7s 261us/sample - loss: 0.5522 - accuracy: 0.7283
Epoch 3/150
25724/25724 [==============================] - 4s 161us/sample - loss: 0.5492 - accuracy: 0.7318
Epoch 4/150
25724/25724 [==============================] - 5s 212us/sample - loss: 0.5468 - accuracy: 0.7324
Epoch 5/150
25724/25724 [==============================] - 4s 172us/sample - loss: 0.5459 - accuracy: 0.7339
Epoch 6/150
25724/25724 [==============================] - 4s 167us/sample - loss: 0.5446 - accuracy: 0.7345
Epoch 7/150
25724/25724 [==============================] - 4s 173us/sample - loss: 0.5433 - accuracy: 0.7341
Epoch 8/150
25724/25724 [==============================] - 4s 156us/sample - loss: 0.5436 - accuracy: 0.7350
Epoch 9/150
25724/25724 [==============================] - 4s 150us/sample - loss: 0.5425 - accuracy: 0.7

In [28]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test_array,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/1 - 1s - loss: 0.5354 - accuracy: 0.7290
Loss: 0.566412414392299, Accuracy: 0.7289795875549316


### Observation on optimization trial #3
Changing the activation function of the second hidden layer from relu to tanh lead to mixed results (slightly lower accuracy, lower loss) than the best performing model: trial #2.

- - -

## Model optimization attempt #4:
#### Retain changes made in trial #3; increase number of epochs in the training phase from 150 to 300

In [29]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn4 = tf.keras.models.Sequential()

nn4.add(tf.keras.layers.Dense(units=116, activation="relu", input_dim=116))

# First hidden layer
nn4.add(tf.keras.layers.Dense(units=58, activation="relu"))

# Second hidden layer
nn4.add(tf.keras.layers.Dense(units=58, activation="tanh"))

# Output layer
nn4.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn4.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 116)               13572     
_________________________________________________________________
dense_14 (Dense)             (None, 58)                6786      
_________________________________________________________________
dense_15 (Dense)             (None, 58)                3422      
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 59        
Total params: 23,839
Trainable params: 23,839
Non-trainable params: 0
_________________________________________________________________


In [30]:
# Compile the model
nn4.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [31]:
# Train the model
fit_model1 = nn4.fit(X_train_scaled, y_train_array, epochs=300)

Train on 25724 samples
Epoch 1/300
25724/25724 [==============================] - 5s 189us/sample - loss: 0.5656 - accuracy: 0.7199
Epoch 2/300
25724/25724 [==============================] - 7s 287us/sample - loss: 0.5531 - accuracy: 0.7289
Epoch 3/300
25724/25724 [==============================] - 4s 173us/sample - loss: 0.5494 - accuracy: 0.7318
Epoch 4/300
25724/25724 [==============================] - 4s 168us/sample - loss: 0.5483 - accuracy: 0.7304
Epoch 5/300
25724/25724 [==============================] - 5s 200us/sample - loss: 0.5459 - accuracy: 0.7325
Epoch 6/300
25724/25724 [==============================] - 4s 158us/sample - loss: 0.5447 - accuracy: 0.7340
Epoch 7/300
25724/25724 [==============================] - 4s 162us/sample - loss: 0.5439 - accuracy: 0.7338
Epoch 8/300
25724/25724 [==============================] - 4s 164us/sample - loss: 0.5429 - accuracy: 0.7353
Epoch 9/300
25724/25724 [==============================] - 4s 162us/sample - loss: 0.5422 - accuracy: 0.7

In [32]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn4.evaluate(X_test_scaled,y_test_array,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/1 - 1s - loss: 0.5277 - accuracy: 0.7291
Loss: 0.5598632306712015, Accuracy: 0.7290962338447571


### Observation on optimization trial #4
Increasing the number of epochs during the training phase lead to improved peformance over trial #3 (higher accuracy, lower loss), with slightly lower accuracy than trial #2 (72.91% vs 72.99%).

In [33]:
# Export model from trial #4 to HDF5 file
nn4.save("AlphabetSoupCharity_Optimization.h5")